In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 8 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Méthode à pas de temps adaptatif
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#

# Méthode à pas de temps adaptatif

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
import warnings
warnings.filterwarnings('ignore')

## Methode basée sur méthode de la “règle 3/8” (ordre 4)

In [ ]:
class embedded_ode_result:

    def __init__(self, y, t, nfev, t_rej, dt, dt_rej, loc_err_est, loc_err):
        self.y = y
        self.t = t
        self.nfev = nfev 
        self.t_rej = t_rej
        self.dt = dt
        self.dt_rej = dt_rej
        self.loc_err_est = loc_err_est
        self.loc_err = loc_err

def rk_embedded(tini, tend, yini, fcn, tol, dt_ini=1.e-2):

    # butcher table of the "3/8 rule" 
    c2 = 1/3; c3 = 2/3; c4 = 1.

    a21 = 1/3; 
    a31 = -1/3; a32 = 1.
    a41 = 1.;   a42 = -1; a43 = 1.

    b1 = 1/8; b2 = 3/8; b3 = 3/8; b4 = 1/8

    # coefficients bhat of the embedded method
    b1hat = 2*b1 - 1/6 
    b2hat = 2*(1-c2)*b2 
    b3hat = 2*(1-c3)*b3
    b4hat = 0.
    b5hat = 1/6
     
    # init all parameters    
    dt = dt_ini

    yini = np.atleast_1d(yini)
    neq = yini.size

    # list
    t_acc = [tini]
    t_rej = []
    dt_acc = []
    dt_rej = []
    y = [yini] 
    loc_err_est = [0.]
    loc_err = [0.]

    tn = tini
    yn = yini

    k5 = np.array(fcn(tn, yn))
    nfev = 1

    while (tn < tend):

        # yn compute with the "3/8 rule" method
        k1 = k5
        k2 = fcn(tn + c2*dt, yn + dt*a21*k1)
        k3 = fcn(tn + c3*dt, yn + dt*(a31*k1 + a32*k2))
        k4 = fcn(tn + c4*dt, yn + dt*(a41*k1 + a42*k2 + a43*k3))

        ynp1 = yn + dt*(b1*k1 + b2*k2 + b3*k3 + b4*k4)

        k5 = np.array(fcn(tn+dt, ynp1))
        nfev += 4

        # yn compute with the embeddded method
        yhatnp1 = yn + dt*(b1hat*k1 + b2hat*k2 + b3hat*k3 + b4hat*k4 + b5hat*k5)

        # compute local error estimation 
        err = np.sqrt(1./neq) * np.linalg.norm((ynp1 - yhatnp1)/(1+np.maximum(np.abs(yn), np.abs(ynp1))))
        dtopt = dt * min(5, max(0.2, 0.9*np.power(tol/err, 1./4.)))

        if (err < tol):

            # compute and save exact local error
            sol = solve_ivp(fcn, (tn, tn+dt), yn, method="DOP853", rtol=1.e-12, atol=1.e-12)
            loc_err.append(np.sqrt(1./neq) * np.linalg.norm(ynp1 - sol.y[:,-1]))
            
            # update for next step
            tn = tn + dt
            yn = ynp1

            # save time, solution and local error estimation
            t_acc.append(tn)
            dt_acc.append(dt) 
            y.append(yn)
            loc_err_est.append(err)

            # compute next dt
            dt = min(dtopt, tend-tn)

        else:
            t_rej.append(tn)
            dt_rej.append(dt) 
            dt = dtopt
            k5 = k1

    y = np.array(np.transpose(np.array(y)), order='F')

    return embedded_ode_result(y, np.array(t_acc), nfev,  np.array(t_rej), np.array(dt_acc), np.array(dt_rej),
                               np.array(loc_err_est), np.array(loc_err))

## Résultats sur le modèle du Brusselator

La dynamique de la réaction oscillante découverte par Belousov et Zhabotinsky, peut être modélisée par le modèle dit de Brusselator en fonction de deux paramètres $a$ et $b$ :

$$
\left\{\begin{aligned}
d_t y_1 & = a - (b+1) y_1 + y_1^2y_2\\
d_t y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b):
        self.a = a
        self.b = b

    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        y1_dot = a - (b+1)*y1 + y1*y1*y2
        y2_dot = b*y1 - y1*y1*y2
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        return np.array([[2*y1*y2 -(b+1), y1*y1], 
                         [-2*y1*y2 + b, -y1*y1] ])

In [ ]:
tini = 0.
tend = 20.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

tol = 1e-2
sol = rk_embedded(tini, tend, yini, fcn, tol)
sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1.e-14, atol=1.e-14, t_eval=sol.t)

glob_err = np.empty(sol.t.size)
for i in range(sol.t.size):
    glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol.y[:,i])


print("   Number of time step : " + str(sol.t.size-1))
print("   Number of function evaluations : " + str(sol.nfev))

fig = make_subplots(rows=3, cols=1, vertical_spacing=0.15)

color = fig.layout.template.layout.colorway
marker_y1 = dict(size=5, symbol='x-thin', line=dict(width=1, color=color[0]))
line_y1 = dict(color=color[0])
marker_y2 = dict(size=5, symbol='x-thin', line=dict(width=1, color=color[1]))
line_y2 = dict(color=color[1])

marker_3 = dict(size=5, symbol='square', color=color[6])

fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='y1', mode='lines+markers', marker=marker_y1, line=line_y1, 
              legendgroup='sol'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name='y2', mode='lines+markers', marker=marker_y2, line=line_y2, 
              legendgroup='sol'), row=1, col=1)

fig.add_trace(go.Scatter(x=sol.t[:-1], y=sol.dt, name='accepted dt', mode='lines+markers', marker_symbol='x', 
                         legendgroup='dt'), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t_rej, y=sol.dt_rej, name='rejected dt', mode='markers', marker_symbol='square', 
                         legendgroup='dt'), row=2, col=1)


fig.add_trace(go.Scatter(x=sol.t, y=sol.loc_err_est, name='est. loc. error', mode='lines+markers', 
                         marker_symbol='x', legendgroup='err'), row=3, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=sol.loc_err, name='loc. error', mode='markers', marker_symbol='square', 
                         legendgroup='err'), row=3, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=glob_err, name='glob. error', mode='markers', marker_symbol='circle', 
                         legendgroup='err'), row=3, col=1)
fig.add_hline(y=tol, line_width=2, line_dash="dot", row=3, col=1)

fig.update_layout(legend=dict(tracegroupgap=275), height=1000)
fig.update_xaxes(title='t', col=1)
fig.update_yaxes(type='log', exponentformat='e', row=2)
fig.update_yaxes(type='log', exponentformat='e', row=3, range=[np.log10(tol)-4, np.log10(tol)+2])
fig.show()

## Coût et précision

In [ ]:
class ode_result:
    def __init__(self, y, t, nfev):
        self.y = y
        self.t = t
        self.nfev = nfev 


def rk38(tini, tend, nt, yini, fcn):
    
    # butcher table of the "3/8 rule" 
    c2 = 1/3; c3 = 2/3; c4 = 1.

    a21 = 1/3; 
    a31 = -1/3; a32 = 1.
    a41 = 1.;   a42 = -1; a43 = 1.

    b1 = 1/8; b2 = 3/8; b3 = 3/8; b4 = 1/8

    # coefficients bhat of the embedded method
    b1hat = 2*b1 - 1/6 
    b2hat = 2*(1-c2)*b2 
    b3hat = 2*(1-c3)*b3
    b4hat = 0.
    b5hat = 1/6

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini
    
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + c2*dt, yn + dt*a21*k1)
        k3 = fcn(tn + c3*dt, yn + dt*(a31*k1 + a32*k2))
        k4 = fcn(tn + c4*dt, yn + dt*(a41*k1 + a42*k2 + a43*k3))
        y[:,it+1] = yn + dt*(b1*k1 + b2*k2 + b3*k3 + b4*k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

In [ ]:
tini = 0.
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = [1001, 2001, 5001, 10001, 20001, 50001]
fe_rk38=[]; norm_rk38=[]

for nt_i in nt:
    t = np.linspace(tini,tend,nt_i)
    rtol=1e-14; atol=1e-14
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=rtol, atol=atol, t_eval=t)
    sol_rk38 = rk38(tini, tend, nt_i, yini, fcn)
    fe_rk38.append(sol_rk38.nfev)
    norm_rk38.append(np.linalg.norm(sol_exa.y-sol_rk38.y) / np.sqrt(nt_i))
    
tol = [1.e-4, 1.e-6, 1.e-8, 1.e-10, 1e-12, 1e-13]
fe_rk_emb=[]; norm_rk_emb=[]

for tol_i in tol:
    sol_rk_emb = rk_embedded(tini, tend, yini, fcn, tol_i)
    rtol=1e-14; atol=1e-14
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=rtol, atol=atol, t_eval=sol_rk_emb.t)
    fe_rk_emb.append(sol_rk_emb.nfev)
    err = 0 
    for i in range(sol_rk_emb.dt.size):
        ldt = sol_rk_emb.dt[i]
        err += ldt * ((sol_exa.y[0,i+1]-sol_rk_emb.y[0,i+1])*(sol_exa.y[0,i+1]-sol_rk_emb.y[0,i+1]) + \
                      (sol_exa.y[1,i+1]-sol_rk_emb.y[1,i+1])*(sol_exa.y[1,i+1]-sol_rk_emb.y[1,i+1])  )
    err = np.sqrt(err)    
    norm_rk_emb.append(err)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=fe_rk38, y=norm_rk38, name='règle 3/8', marker_symbol='x'))
fig.add_trace(go.Scatter(x=fe_rk_emb, y=norm_rk_emb, name='méthode emboitée', marker_symbol='x'))
fig.update_xaxes(type='log', exponentformat='e', title="Nb d'évaluations de la fonction")
fig.update_yaxes(type='log', exponentformat='e', title="Norme l2 de l'erreur")